# Variante del test de homocedasticidad de White

## Heterocedasticidad en la regresión lineal

Sabemos que en el modelo clásico de regresión lineal, una de las condiciones que se le exige es que los errores sean homocedásticos, es decir, si $\epsilon_i$ es el error de la $i$-ésima observación, y $x_{ij}$ el valor de la $j$-ésima variable explicativa correspondiente al individuo muestral $i$, con $i \in \{1, \ldots, n\}$ y $j \in \{1, \ldots,  k\}$, entonces
$$
Var(\epsilon_i|x_{i1}, \ldots, x_{ik}) = \sigma^2, \forall i, j.
$$

Bajo la condición de homocedasticidad, los errores estándar de los estimadores MCO (mínimos cuadráticos ordinarios) tienen las propiedades necesarias para hacer sobre ellos los contrastes habituales. Si la varianza no es constante, estamos en presencia de heterocedasticidad y los errores estándar pierden sus propiedades, lo que pone en duda la validez de los contrastes al uso, debiendo aplicar entonces métodos robustos.

Se hace entonces necesario echar mano contrastes como el de Breusch-Pagan o el de White para comprobar si los errores son homocedásticos.


## Contrastes de Breusch-Pagan y de White

Puesto que en el modelo de regresión se supone que $E(\epsilon_i|x_{i1}, \ldots, x_{ik}) = 0$, nos queda que
$$
Var(\epsilon_i|x_{i1}, \ldots, x_{ik}) = E(\epsilon_i^2|x_{i1}, \ldots, x_{ik}),
$$
por lo que si es cierto que la varianza no depende de las variables explicativas, en el modelo de regresión
$$
\epsilon_i^2 = \alpha_0 + \alpha_1 x_{i1} + \alpha_2 x_{i2} + \cdots + \alpha_k x_{ik} + e_i,
$$
los coeficientes $\alpha_j$ deberían ser todos nulos. El contraste se plantea en los términos
$$
\left\{
\begin{array}{l}
H_0: \alpha_j = 0, \forall j \\
H_1: \exists j \mbox{ t.q } \alpha_j \neq 0.
\end{array}
\right.
$$
$H_0$ equivale a la homocedasticidad y $H_1$ a la heterocedasticidad. El estadístico de contraste es
$$
LM = n \cdot R^2 \sim \chi^2(k),
$$
siendo $R^2$ el coeficiente de determinación de la regresión de $\epsilon_i^2$ sobre las variables independientes.

R tiene ya este contraste,


In [12]:
# Lectura de datos del texto de Wooldridge
library(wooldridge)
data(gpa1)
datos <- gpa1[,c('colGPA', 'hsGPA', 'ACT', 'alcohol')]
head(datos)

# Ajuste MCO
mod <- lm(colGPA ~ hsGPA + ACT + alcohol, data = datos)
summary(mod)

# Contraste Breusch-Pagan
library(lmtest)
bptest(mod)

,colGPA,hsGPA,ACT,alcohol
,<dbl>,<dbl>,<int>,<dbl>
1,3.0,3.0,21,1.0
2,3.4,3.2,24,1.0
3,3.0,3.6,26,1.0
4,3.5,3.5,27,0.0
5,3.6,3.9,28,1.5
6,3.0,3.4,25,0.0



Call:
lm(formula = colGPA ~ hsGPA + ACT + alcohol, data = datos)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.84870 -0.25571 -0.02872  0.27773  0.83642 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 1.278706   0.342886   3.729  0.00028 ***
hsGPA       0.456741   0.096747   4.721 5.73e-06 ***
ACT         0.008771   0.011030   0.795  0.42787    
alcohol     0.006456   0.021435   0.301  0.76372    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.3414 on 137 degrees of freedom
Multiple R-squared:  0.177,	Adjusted R-squared:  0.1589 
F-statistic: 9.819 on 3 and 137 DF,  p-value: 6.555e-06



	studentized Breusch-Pagan test

data:  mod
BP = 1.1124, df = 3, p-value = 0.7741


Con este resultado, mantendríamos la hipótesis de homocedasticidad.

El contraste de White extiende el de Breusch-Pagan añadiendo a la ecuación de regresión de $\epsilon_i^2$ los términos cuadráticos y todas las interacciones de orden dos, lo que puede hacer que el número total de términos aumente considerablemente.

In [13]:
# Contraste de White
bptest(mod, ~ hsGPA * ACT * alcohol - hsGPA:ACT:alcohol + 
              I(hsGPA^2) + I(ACT^2) + I(alcohol^2), data = datos)


	studentized Breusch-Pagan test

data:  mod
BP = 5.8921, df = 9, p-value = 0.7507


Seguimos manteniendo como válida la homocedasticidad.

## Variante del contraste de White

Wooldridge ([[1]](#1), pág. 272) expone una variante basada en utilizar los valores ajustados
$$
\hat{y}_i = \hat{\beta}_0 + \hat{\beta}_1 x_{i1} + \cdots + \hat{\beta}_k x_{ik},
$$
haciendo el contraste de homocedasticidad para la regresión
$$
\epsilon_i^2 = \alpha_0 + \alpha_1 \hat{y}_i + \alpha_2 \hat{y}_i^2 + e_i.
$$
Se plantea entonces el contraste
$$
\left\{
\begin{array}{l}
H_0: \alpha_1 = \alpha_2 = 0 \\
H_1: \alpha_1 \neq 0 \mbox{ o } \alpha_2 \neq 0,
\end{array}
\right.
$$
que equivale al contraste global de regresión $F$, con 2 y $n-3$ grados de libertad.

In [15]:
# Los residuos y los valores ajustados los tenemos en mod

# Los cuadrados de los residuos son
eps2 <- mod$residuals^2

# Los valores ajustados son
yhat <- mod$fitted.values

# Hacemos la regresión propuesta para la variante de White
summary(lm(eps2 ~ yhat + I(yhat^2)))


Call:
lm(formula = eps2 ~ yhat + I(yhat^2))

Residuals:
     Min       1Q   Median       3Q      Max 
-0.14833 -0.09459 -0.04416  0.04355  0.60560 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)
(Intercept)   3.5414     3.5200   1.006    0.316
yhat         -2.3331     2.3176  -1.007    0.316
I(yhat^2)     0.3953     0.3809   1.038    0.301

Residual standard error: 0.1377 on 138 degrees of freedom
Multiple R-squared:  0.01426,	Adjusted R-squared:  -2.82e-05 
F-statistic: 0.998 on 2 and 138 DF,  p-value: 0.3713


El contraste global de regresión arroja un $p$-valor de 0.3713, por lo que no hay indicios que nos lleven a rehazar la hipótesis nula $H_0: \alpha_1 = \alpha_2 = 0$, resultado compatible con el de Breusch-Pagan y White.

La ventaja de esta variante es que nos permite hacer el contraste de White con mayor número de variables explicativas, sin complicar el código.

## Bibliografía
<a id="1">[1]</a> 
Wooldridge, Jeffrey. (2020)
Introductory Econometrics. A Modern Approach.
7ª Edición. Cengage.
